Hitta kommuner som inte har en öppan data portal i WD
* denna [Notebook](https://github.com/salgo60/open-data-examples/blob/master/PSI%20katalog%20svenska%20kommuner.ipynb)
* [GIST](https://gist.github.com/salgo60/62332ca873087edddb7f58397944e62e)


In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2022-06-24 11:51:02.432391


In [2]:
import sys
import  urllib3       

from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"
http = urllib3.PoolManager()

#
# SPARQL https://w.wiki/5LSY 
# Kommuner utan öppna data portal
query = """SELECT DISTINCT ?item ?wdLabel ?url ?PSIdata WHERE {
  ?item wdt:P31 wd:Q127448.
  OPTIONAL { ?item wdt:P856 ?url. }
  minus { ?item wdt:P8402 ?openDataPortal. }
  OPTIONAL { ?item (wdt:P8402/wdt:P856) ?PSIdata. }
  OPTIONAL {
    ?item rdfs:label ?wdLabel.
    FILTER((LANG(?wdLabel)) = "sv")
  }
  FILTER(?item != wd:Q42009)
}
ORDER BY desc(?wdLabel)
"""


def get_results(endpoint_url, query):
    user_agent = "user Salgo50/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


''' Check if URLS is ok'''
def check(url):
    try:
        r = http.request('GET', url)
        
        #print("\tStatus: ", r.status, " \t", url, " \t",print(r.headers))
        try:
            print("\t\tX-TEC-API-ROOT: ",r.headers['X-TEC-API-ROOT'])
        except:
            pass
        if r.status == 200:
                print("Status: ",r.status, " \t", url)

        if r.status == 404:
            return False
        return True
    except:
        pass
    return False

def getUrl(inUrl):
    if inUrl.endswith("/"):
        return inUrl
    else:
        return inUrl+"/"
    pass


def checkResult(results):
    global ok, notok , listSites
    for result in results["results"]["bindings"]:
        #print(result)
        currSite = {}
        #currSite["URL"] = getUrl(result["url"]["value"]) + "PSIdata"
        currSite["URL"] = getUrl(result["url"]["value"]) + "psi"
        currSite["item"] = result["item"]["value"].replace("http://www.wikidata.org/entity/","")
        currSite["name"] = result["wdLabel"]["value"]
        currSite["URLStatus"] = check(currSite["URL"])
        try:
            currSite["PSIdata"] = result["PSIdata"]["value"]
            currSite["PSIdataStatus"] = check(result["PSIdata"]["value"])
        except:
            pass
        listSites.append(currSite)
    return

listSites = []
results = get_results(endpoint_url, query)
checkResult(results)


def checkOKPSIData(listSites):
    ok = 0
    notok = 0
    for site in listSites:
        if site["URLStatus"]:
            ok += 1
        else:
            notok += 1
    return ok, notok


ok, notok  = checkOKPSIData(listSites)

print("OK: ",ok,"\t not ok",notok, "Antal utan Öppen data portal enl. Wikidata: ", len(listSites))

Status:  200  	 http://www.are.se/psi
Status:  200  	 http://www.vastervik.se/psi
Status:  200  	 http://www.torsas.se/psi
Status:  200  	 http://www.tibro.se/psi
Status:  200  	 https://nykoping.se/psi
Status:  200  	 https://www.karlsborg.se/psi
Status:  200  	 http://www.hjo.se/psi
Status:  200  	 http://www.danderyd.se/psi
Status:  200  	 http://www.boden.se/psi
OK:  16 	 not ok 145 Antal utan Öppen data portal enl. Wikidata:  161


In [3]:
#listSites

In [4]:
end = datetime.now()
print("Ended: ", end) 
print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Ended:  2022-06-24 11:51:51.336025
Time elapsed (hh:mm:ss.ms) 0:00:48.904711
